# Hammond Landforms: Neighborhood Analysis Window

In [ ]:
# builtins
import pathlib

# externals
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# locals
from pysegcnn.core.utils import img2np
from ai4ebv.core.landforms import HAMMOND_LANDFORM_CLASSES, SAYRE_LANDFORM_CLASSES, SayreLandforms

In [ ]:
# neighborhood analysis window sizes in km
NAW_SIZES = [1, 2]

In [ ]:
# path to Hammond landform layers
PATH_LAYERS = pathlib.Path('/mnt/CEPH_PROJECTS/AI4EBV/LANDFORMS/Python/')

In [ ]:
# list of possible slope, relief, and profile combinations
POSSIBLE_COMBINATIONS = [s + r + p for s in [100, 200, 300, 400] for r in [10, 20, 30, 40, 50, 60] for p in [0, 1, 2, 3, 4]]

In [ ]:
def class_distribution(array, class_scheme, no_data=9999, freq=True):
    # compute class distribution
    classes, counts = np.unique(array, return_counts=True)
    
    # remove NoData class
    mask = classes != no_data
    classes = classes[mask]
    counts = counts[mask]
    counts = counts / counts.sum() if freq else counts  # whether to convert count to frequency
    
    # get classes which are defined
    defined = np.isin(classes, class_scheme)
    color = np.where(defined, 'g', 'r')
    
    # create DataFrame for plotting purpose
    class_dist = pd.DataFrame(np.array([classes, counts]).transpose(), columns=['class', 'count'])
    class_dist['color'] = color
    
    # fill not defined/computed classes
    if class_scheme is HAMMOND_LANDFORM_CLASSES:
        for c in POSSIBLE_COMBINATIONS:
            if c not in class_dist['class'].values:
                class_dist = class_dist.append(pd.DataFrame([[c, 0, 'r']], columns=['class', 'count', 'color']))
            
    # remove impossible combinations
    for idx, row in class_dist.iterrows():
        if class_scheme is HAMMOND_LANDFORM_CLASSES:
            if row['class'] not in POSSIBLE_COMBINATIONS:
                class_dist = class_dist.drop(idx)
            
    # sort by class id
    class_dist = class_dist.sort_values('class')
    
    return class_dist

## Load Hammond landform layers

In [ ]:
# load Hammond landform layers generated with different NAW sizes
hlf = {naw: img2np(PATH_LAYERS.joinpath('naw-{:d}-km/SRTM_alps_030m_03035_HLF_naw-{:d}km.tif'
                                        .format(naw, naw))) for naw in NAW_SIZES}

In [ ]:
# load WTE landform layers generated with different NAW sizes
wte = {naw: img2np(PATH_LAYERS.joinpath('naw-{:d}-km/SRTM_alps_030m_03035_WTE-LF_naw-{:d}km.tif'
                                        .format(naw, naw))) for naw in NAW_SIZES}

## Class distributions

In [ ]:
# compute Hammond landform classes and counts depending on NAW size
hlf_classes = {k: class_distribution(v, HAMMOND_LANDFORM_CLASSES) for k, v in hlf.items()}

In [ ]:
# compute WTE landform classes and counts depending on NAW size
wte_classes = {k: class_distribution(v, SAYRE_LANDFORM_CLASSES) for k, v in wte.items()}

### Hammond landforms class distribution

In [ ]:
# plot class distributions
fig, axes = plt.subplots(len(NAW_SIZES), 1, figsize=(24, int(5 * len(NAW_SIZES))), sharex=True)
axes = axes.flatten()

# iterate over NAW sizes
for naw, ax in zip(NAW_SIZES, axes):
    # plot class distribution
    sns.barplot(x='class', y='count', data=hlf_classes[naw], ax=ax, palette=hlf_classes[naw]['color'])
    ax.set_ylim(0, 0.2)
    ax.set_xlabel('')
    ax.set_ylabel('Frequency', fontsize=18, labelpad=10)
    ax.set_title('Circular NAW radius: {:d} km'.format(naw), fontsize=18, pad=10)
    ax.set_xticks(np.arange(0, len(POSSIBLE_COMBINATIONS)))
    ax.set_xticklabels(POSSIBLE_COMBINATIONS)
    
# axes tick parameters
for ax in axes:
    ax.tick_params('both', labelsize=14)
    ax.tick_params('x', labelrotation=90)

# set shared xtick labels
axes[-1].set_xlabel('Hammond landform classes', fontsize=18, labelpad=10);

# add legend
patches = [mpatches.Patch(color=c, label='defined' if c == 'g' else 'not defined') for c in ['g', 'r']]
axes[-1].legend(handles=patches, loc=1, frameon=False, fontsize=18);

# save figure
fig.subplots_adjust(hspace=0.15)
fig.savefig('./Figures/HLF_class_distribution.png', dpi=300, bbox_inches='tight')

### Class distribution depending on NAW-size

In [ ]:
for k, v in hlf_classes.items():
    v['naw'] = k
hlf_classes = pd.concat(hlf_classes.values(), ignore_index=True)

In [ ]:
for k, v in wte_classes.items():
    v['naw'] = k
wte_classes = pd.concat(wte_classes.values(), ignore_index=True)

In [ ]:
# plot class distributions
fig, ax = plt.subplots(1, 1, figsize=(24, 5))

# plot class distribution
sns.barplot(x='class', y='count', hue='naw', data=hlf_classes, ax=ax, palette=sns.color_palette('mako', 2))
ax.set_ylim(0, 0.2)
ax.set_xlabel('')
ax.set_ylabel('Frequency', fontsize=18, labelpad=10)
ax.set_xticks(np.arange(0, len(POSSIBLE_COMBINATIONS)))
ax.set_xticklabels(POSSIBLE_COMBINATIONS)
    
# axes tick parameters
ax.tick_params('both', labelsize=14)
ax.tick_params('x', labelrotation=90)

# add legend
h, _ = ax.get_legend_handles_labels()
ax.get_legend().remove()
ax.legend(h, ['NAW: {:d}km'.format(naw) for naw in NAW_SIZES], frameon=False, fontsize=18)

# set shared xtick labels
ax.set_xlabel('Hammond landform classes', fontsize=18, labelpad=10);
fig.savefig('./Figures/HLF_class_distribution_naw.png', dpi=300, bbox_inches='tight')

In [ ]:
# plot class distributions
fig, ax = plt.subplots(1, 1, figsize=(24, 5))

# plot class distribution
sns.barplot(x='class', y='count', hue='naw', data=wte_classes, ax=ax, palette=sns.color_palette('mako', 2))
ax.set_ylim(0, 0.5)
ax.set_xlabel('')
ax.set_ylabel('Frequency', fontsize=18, labelpad=10)
ax.set_xticks(np.arange(0, len(SAYRE_LANDFORM_CLASSES)))
ax.set_xticklabels([SayreLandforms.label_dict()[k]['label'] for k in SAYRE_LANDFORM_CLASSES])
    
# axes tick parameters
ax.tick_params('both', labelsize=14)
ax.tick_params('x', labelrotation=90)

# add legend
h, _ = ax.get_legend_handles_labels()
ax.get_legend().remove()
ax.legend(h, ['NAW: {:d}km'.format(naw) for naw in NAW_SIZES], frameon=False, fontsize=18)

# set shared xtick labels
ax.set_xlabel('WTE landform classes', fontsize=18, labelpad=10);
fig.savefig('./Figures/WTE_class_distribution.png', dpi=300, bbox_inches='tight')